In [1]:
# Import dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages

In [2]:
# Import the input dataset.
file_path = "Resources/charity_data.csv"
charity_df = pd.read_csv(file_path)
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Get all columns
charity_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [6]:
# Check for null values
charity_df.isnull().sum()

EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [7]:
# Check the unique value counts to see if binning is required
ask_amounts = charity_df.ASK_AMT.value_counts()
ask_amounts

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [8]:
# Determine max / min values of ask amount
charity_df.ASK_AMT.min()

5000

In [9]:
# Determine max / min values of ask amount
charity_df.ASK_AMT.max()

8597806340

In [10]:
charity_df.ASK_AMT.describe()

count    3.429900e+04
mean     2.769199e+06
std      8.713045e+07
min      5.000000e+03
25%      5.000000e+03
50%      5.000000e+03
75%      7.742000e+03
max      8.597806e+09
Name: ASK_AMT, dtype: float64

In [11]:
# Determine which values to replace
replace_ask_amount = list(ask_amounts[ask_amounts < 5000].index)

# Replace in DataFrame
for amount in replace_ask_amount:
    charity_df.ASK_AMT = charity_df.ASK_AMT.replace(amount,"Other")


# Check to make sure binning was successful
charity_df.ASK_AMT.value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [12]:
# Encode ASK_AMT
charity_df = pd.get_dummies(charity_df, columns=['ASK_AMT'])
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,1,1,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,0,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,1,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,0,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,0,1


In [13]:
# Check the unique value counts to see if binning is required
charity_df.IS_SUCCESSFUL.value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [14]:
# Check the unique value counts to see if binning is required
charity_df.STATUS.value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [15]:
# Drop rows where STATUS = 0
charity_df = charity_df[charity_df['STATUS'] == 1]
charity_df.STATUS.value_counts()

1    34294
Name: STATUS, dtype: int64

In [17]:
# Check the unique value counts to see if binning is required
application_types = charity_df.APPLICATION_TYPE.value_counts()
application_types

T3     27032
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Determine which values to replace
replace_application_type = list(application_types[application_types < 1000].index)

# Replace in DataFrame
for app_type in replace_application_type:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app_type,"Other")

# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27032
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [20]:
# Encode APPLICATION_TYPE
charity_df = pd.get_dummies(charity_df, columns=['APPLICATION_TYPE'])
charity_df.head()

,EIN,NAME,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,Independent,C1000,ProductDev,Association,1,0,N,1,1,0,1,0,0,0,0,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,0,1,0,0,1,0,0,0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,1,0,0,0,0,0,1,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,0,1,0,0,1,0,0,0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,0,1,0,0,1,0,0,0


In [22]:
# Check the unique value counts to see if binning is required
affiliations = charity_df.AFFILIATION.value_counts()
affiliations

Independent         18477
CompanySponsored    15703
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [23]:
# Determine which values to replace
replace_affiliations = list(affiliations[affiliations < 15000].index)

# Replace in DataFrame
for affiliation in replace_affiliations:
    charity_df.AFFILIATION = charity_df.AFFILIATION.replace(affiliation,"Other")

# Check to make sure binning was successful
charity_df.AFFILIATION.value_counts()

Independent         18477
CompanySponsored    15703
Other                 114
Name: AFFILIATION, dtype: int64

In [25]:
# Drop rows where AFFILIATION = 'Other'
charity_df = charity_df[charity_df['AFFILIATION'] != 'Other']
charity_df.AFFILIATION.value_counts()

Independent         18477
CompanySponsored    15703
Name: AFFILIATION, dtype: int64

In [26]:
# Check the unique value counts to see if binning is required
classifications = charity_df.CLASSIFICATION.value_counts()
classifications

C1000    17256
C2000     6051
C1200     4825
C3000     1917
C2100     1880
         ...  
C4200        1
C2170        1
C1370        1
C1236        1
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [27]:
# Determine which values to replace
replace_classifications = list(classifications[classifications < 1800].index)

# Replace in DataFrame
for classification in replace_classifications:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification,"Other")

# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17256
C2000     6051
C1200     4825
Other     2251
C3000     1917
C2100     1880
Name: CLASSIFICATION, dtype: int64

In [28]:
# Encode CLASSIFICATION
charity_df = pd.get_dummies(charity_df, columns=['CLASSIFICATION'])
charity_df.head()

,EIN,NAME,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_5000,...,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,Independent,ProductDev,Association,1,0,N,1,1,...,0,0,0,0,1,0,0,0,0,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,Independent,Preservation,Co-operative,1,1-9999,N,1,0,...,1,0,0,0,0,0,1,0,0,0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,CompanySponsored,ProductDev,Association,1,0,N,0,1,...,0,0,1,0,0,0,0,0,1,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,CompanySponsored,Preservation,Trust,1,10000-24999,N,1,0,...,1,0,0,0,0,0,1,0,0,0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,Independent,Heathcare,Trust,1,100000-499999,N,1,0,...,1,0,0,0,1,0,0,0,0,0


In [29]:
# Check the unique value counts to see if binning is required
use_cases = charity_df.USE_CASE.value_counts()
use_cases

Preservation     27994
ProductDev        5659
CommunityServ      381
Heathcare          143
Other                3
Name: USE_CASE, dtype: int64

In [30]:
# Determine which values to replace
replace_use_cases = list(use_cases[use_cases < 5000].index)

# Replace in DataFrame
for use_case in replace_use_cases:
    charity_df.USE_CASE = charity_df.USE_CASE.replace(use_case,"Other")

# Check to make sure binning was successful
charity_df.USE_CASE.value_counts()

Preservation    27994
ProductDev       5659
Other             527
Name: USE_CASE, dtype: int64

In [31]:
# Drop rows where USE_CASE = 'Other'
charity_df = charity_df[charity_df['USE_CASE'] != 'Other']
charity_df.USE_CASE.value_counts()

Preservation    27994
ProductDev       5659
Name: USE_CASE, dtype: int64

In [32]:
# Check the unique value counts to see if binning is required
organizations = charity_df.ORGANIZATION.value_counts()
organizations

Trust           23053
Association     10081
Co-operative      482
Corporation        37
Name: ORGANIZATION, dtype: int64

In [33]:
# Determine which values to replace
replace_organizations = list(organizations[organizations < 10000].index)

# Replace in DataFrame
for org_type in replace_organizations:
    charity_df.ORGANIZATION = charity_df.ORGANIZATION.replace(org_type,"Other")

# Check to make sure binning was successful
charity_df.ORGANIZATION.value_counts()

Trust          23053
Association    10081
Other            519
Name: ORGANIZATION, dtype: int64

In [34]:
# Drop rows where ORGNAZATION = 'Other'
charity_df = charity_df[charity_df['ORGANIZATION'] != 'Other']
charity_df.ORGANIZATION.value_counts()

Trust          23053
Association    10081
Name: ORGANIZATION, dtype: int64

In [35]:
# Check the unique value counts to see if binning is required
incomes = charity_df.INCOME_AMT.value_counts()
incomes

0                23804
25000-99999       3634
100000-499999     3251
1M-5M              847
1-9999             692
10000-24999        513
10M-50M            174
5M-10M             146
50M+                73
Name: INCOME_AMT, dtype: int64

In [ ]:
# NOT SURE WHAT TO DO ABOUT INCOME... $0????

In [36]:
# Check the unique value counts to see if binning is required
considerations = charity_df.SPECIAL_CONSIDERATIONS.value_counts()
considerations

N    33108
Y       26
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [37]:
# Drop rows where SPECIAL_CONSIDERATIONS = Y
charity_df = charity_df[charity_df['SPECIAL_CONSIDERATIONS'] == "N"]
charity_df.SPECIAL_CONSIDERATIONS.value_counts()

N    33108
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [38]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      18770
AFFILIATION                   2
USE_CASE                      2
ORGANIZATION                  2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        1
dtype: int64

In [39]:
# drop unwanted columns
charity_df=charity_df.drop(['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'], axis=1)
charity_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other
0,Independent,ProductDev,Association,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0
2,CompanySponsored,ProductDev,Association,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
3,CompanySponsored,Preservation,Trust,10000-24999,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0
5,Independent,Preservation,Trust,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0
6,Independent,Preservation,Trust,100000-499999,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0


In [40]:
# Regenerate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

AFFILIATION     2
USE_CASE        2
ORGANIZATION    2
INCOME_AMT      9
dtype: int64

In [41]:
# Create a OneHotEncoder instance.
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder  using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame.
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Independent,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Merge one-hot encoded features and drop the originals.
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,0,1,0,0,0,0,0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,0,1,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,1,0,0,1,0,0,0,0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1,1,0,0,0,1,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1,0,1,0,0,1,0,0,0,1,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# Split the preprocessed data into features and target arrays.
y = charity_df['IS_SUCCESSFUL'].values
X = charity_df.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [44]:
# Create a StandardScaler instance.
scaler = StandardScaler()

# Fit the StandardScaler.
X_scaler = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [45]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# Add first and second hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 240       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________
